In [1]:
import models.models_multi_task as md_multi
import datasets.iemocap as ds
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

# Проба многозадачного обучения IEMOCAP

## Модель:

Подготовил следующую архитектуру, основанную на AlexNet:

In [2]:
model = md_multi.AlexNetMultiTask(num_emotions=4, num_speakers=10, num_genders=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = model.to(device)
print(model)
summary(model, (1, 224, 224), batch_size=256)
model=None
torch.cuda.empty_cache()

AlexNetMultiTask(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
  (joint_classifier): Sequential(
    (0): Linear(in_features=6400, out_features=2048, bias=True)
    (1): Dropout(p=0.5, i

## Проверяю модель и свой код для обучения на работоспособность

Мой код для обучения построен вокруг класса TrainingSession в модуле multitask_training_session.py <br>
У этого класса есть метод overfit_one_batch, этот метод позволяет проверить модель на малой выборке данных, это полезно для того, чтобы посмотреть, как модель переобучается. 
Если она переобучается хорошо, то хотя бы градиентный спуск работает нормально.

In [2]:
iemocap_224_noprep_train = ds.IemocapDataset(  # Без препроцессинга, тренировочный
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)
iemocap_224_noprep_test = ds.IemocapDataset(  # Без препроцессинга, тестовый
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


In [3]:
train_ds = iemocap_224_noprep_train
test_ds = iemocap_224_noprep_test

In [4]:
model = md_multi.AlexNetMultiTask(4, 10, 2)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [5]:
ts = TrainingSession(name='FirstTry',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=100,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER)

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION FirstTry__IEMOCAP-4_four_prep-false_224_train INITIALIZED
Trying to load checkpoint from file
File not found, starting from scratch...


In [6]:
ts.overfit_one_batch(100, 100)

Epoch #0
TRAIN SIZE 2858
TEST SIZE 714
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
# Time passed: 13 s
# Epoch losses | emotion = 2867.4915 | speaker = 2096.9410 | gender = 883.6844 |
# Train accuracies | emotion = 0.3266 | speaker = 0.159 | gender = 0.5322 |
Epoch #1
TRAIN SIZE 2858
TEST SIZE 714
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
# Time passed: 12 s
# Epoch losses | emotion = 1.3458 | speaker = 2.2573 | gender = 0.6891 |
# Train accuracies | emotion = 0.3508 | speaker = 0.1798 | gender = 0.547 |
Epoch #2
TRAIN SIZE 2858
TEST SIZE 714
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
# Time passed: 10 s
# Epoch losses | emotion = 1.3454 | speaker = 2.2579 | gender = 0.6881 |
# Train accuracies | emo

KeyboardInterrupt: 